In [ ]:
!pip install --upgrade pip

# ---- Streamlit & Ngrok ----
!pip install streamlit
!pip install pyngrok

# ---- Google Gemini API ----
!pip install google-generativeai

# ---- NLP / RAG ----
!pip install sentence-transformers
!pip install langchain
!pip install langchain-community
!pip install chromadb

# ---- image processing ----
!pip install pillow
!pip install wordfreq

# ---- assistence ----
!pip install pandas numpy
!pip install requests

# ---- Optional: Improve how images are displayed in Streamlit ----
!pip install streamlit-extras


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 22.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 22.5 MB/s eta 0:00:00
  Attempting uninstall: pip
    Found existing installation: pip 24.1.2
    Uninstalling pip-24.1.2:
  Attempting uninstall: pip
    Found existing installation: pip 24.1.2
    Uninstalling pip-24.1.2:
      Successfully uninstalled pip-24.1.2
      Successfully uninstalled pip-24.1.2
   ━━━━━━╸━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/9.0 MB 9.0 MB/s eta 0:00:01Requirement already satisfied: six>=1.5 in /usr/local/lib/python3.12/dist-packages (from python-dateutil>=2.8.2->pandas<3,>=1.4.0->streamlit) (1.17.0)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.0/9.0 MB 17.7 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.0/9.0 MB 17.7 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 44.0 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 44.0 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

KeyboardInterrupt: 

KeyboardInterrupt: 

In [ ]:
!apt-get install -y fonts-noto-cjk fonts-noto-color-emoji

In [ ]:
# set API Key
from google.colab import userdata
import os

os.environ["GEMINI_API_KEY"] = userdata.get("GEMINI_API_KEY")


In [ ]:
!ls /content/data


In [ ]:
%%writefile app.py
import os
from io import BytesIO
import json
import re
import requests

import streamlit as st
import google.generativeai as genai
from google import genai as genai_new
from google.generativeai import types as genai_types

from PIL import Image as PilImage, ImageDraw, ImageFont
from wordfreq import word_frequency

import pandas as pd
from collections import Counter
from langchain_community.vectorstores import Chroma
from langchain_community.embeddings import HuggingFaceEmbeddings

from google.colab import userdata



# ========== 1. API Key & Model Configuration ==========

api_key = os.environ.get("GEMINI_API_KEY")


if not api_key:
    st.error("❌ GEMINI_API_KEY is not set. Please configure the environment variable before starting Colab.")
    st.stop()

# use gemini 2.5 flash
TEXT_MODEL = "models/gemini-2.5-flash"
IMAGE_MODEL = "gemini-2.5-flash-image"

genai.configure(api_key=api_key)
image_client = genai_new.Client(api_key=api_key)


# ========== 2. Ontology + Chroma (Multi-organ RAG) ==========
# 👉 Please update `csv_path` to your own file path.
#    In Colab, it is recommended to first copy the file to /content,
#    then simply set csv_path = "ontology_multi_organ_steps.csv"
csv_path = "/content/data/ontology_multi_organ_steps.csv"


df_onto = pd.read_csv(csv_path)
print("Ontology loaded successfully, number of rows:", len(df_onto))

texts = (
    df_onto["activity"].astype(str)
    + " | organ: " + df_onto["organ"].astype(str)
    + " | mechanism: " + df_onto["mechanism"].astype(str)
).tolist()

metadata = df_onto.to_dict(orient="records")

embedding_fn = HuggingFaceEmbeddings(
    model_name="sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2"
)

db = Chroma.from_texts(
    texts=texts,
    embedding=embedding_fn,
    metadatas=metadata,
    collection_name="bio_multi_organ"
)

print("Chroma set up successfully")


# ========== 3. RAG Story Generation – 4 Functions ==========
def extract_events_gemini(user_input: str):
    """
    Extract 1–4 physiologically relevant activity events from the user’s input
    using Gemini.

    Returns a list of short English phrases, for example:
        ["drinking coffee", "running", "staying up late"]
    """

    prompt = f"""
You are a biomedical event extractor.

Read the user's description and identify the DISTINCT lifestyle or physiological
activities that have meaningful effects on the body
(e.g., "drinking coffee", "running", "pulling an all-nighter", "eating spicy food").

Rules:
- Return ONLY a JSON array of short English phrases.
- Each phrase should be 2–6 words.
- Include ONLY the most important 1–4 activities that significantly change physiology.
- Do NOT break one coherent activity into many tiny fragments.
- No explanations, no extra text.

User input:
\"\"\"{user_input}\"\"\"
"""

    model = genai.GenerativeModel(TEXT_MODEL)
    response = model.generate_content(prompt)
    text = response.text.strip()

    try:
        events = json.loads(text)
        events = [e.strip() for e in events if isinstance(e, str) and e.strip()]
    except Exception:
        # Fallback: if parsing fails, treat the entire output as a single event
        events = [text]

    if len(events) > 4:
        events = events[:4]

    return events


def build_multi_organ_journey(user_input: str, k_search: int = 20) -> str:
    """
    1. Use Gemini to break the user_input into several events (1–4).
    2. For each event:
        - Retrieve ontology records from Chroma
        - Identify the most relevant activity name
        - Sort by step to build a multi-organ journey
    3. Concatenate all activity journeys to form the scientific backbone
      for the subsequent storyboard and narrative generation.
    """


    events = extract_events_gemini(user_input)
    if not events:
        return "No events detected."

    journey_sections = []
    activity_index = 1

    for ev in events:
        results = db.similarity_search(ev, k=k_search)
        if not results:
            continue

        acts = [r.metadata.get("activity", "unknown") for r in results]
        main_activity, _ = Counter(acts).most_common(1)[0]

        filtered = [r for r in results if r.metadata.get("activity", "unknown") == main_activity]
        filtered = sorted(filtered, key=lambda r: int(r.metadata.get("step", 0)))

        lines = []
        lines.append(f"Activity {activity_index}: {main_activity}")
        lines.append(f"User-described event phrase: {ev}")
        lines.append("Multi-organ journey:")
        for r in filtered:
            step = r.metadata.get("step", "?")
            organ = r.metadata.get("organ", "organ")
            mech = r.metadata.get("mechanism", None) or r.page_content
            lines.append(f"- Step {step}: [{organ}] {mech}")

        journey_sections.append("\n".join(lines))
        activity_index += 1

    if not journey_sections:
        return "No matched mechanisms in ontology."

    return "\n\n".join(journey_sections)


def generate_scientific_storyboard(user_input: str) -> str:
    """
    Generate a 4-scene storyboard based on the multi-organ journey.
    Language: match the user's input language as closely as possible.
    """


    journey = build_multi_organ_journey(user_input, k_search=20)

    prompt = f"""
You are a Biological Science Director planning an educational comic/anime episode
about what happens inside the human body.

You will be given one or more multi-organ physiological journeys for the user's activities.

Your job: convert them into a 4-scene scientific storyboard for ONE coherent adventure.

Language:
- Use the same primary language as the user's input below.
- If the user's input is mainly Chinese, write the storyboard in Chinese.
- If it is mainly English, write it in English.

STRUCTURE (MANDATORY):
You MUST use EXACTLY these four headings (in English, keep them as titles):

**SCENE 1: ENTRY**
**SCENE 2: TRANSPORT**
**SCENE 3: THE MECHANISM (CLIMAX)**
**SCENE 4: RESOLUTION**

Under each heading, write 1–3 short paragraphs that describe:
- Where we are (organs, tissues)
- Who appears (molecules, cells, organs, pathogens, drugs, etc.)
- What key mechanisms happen in that scene.

Multi-organ journeys (reference only):

{journey}

User input (for language and context):
\"\"\"{user_input}\"\"\"
"""

    model = genai.GenerativeModel(TEXT_MODEL)
    response = model.generate_content(prompt)
    storyboard = response.text.strip()
    return storyboard


def generate_physiology_story(user_input: str) -> str:
    """
    Generate the final story based on the 4-scene storyboard:
    - The narrative perspective may be chosen by the model (omniscient narrator, a molecule’s POV,
      a single cell, etc.).
    - The story should match the user's input language (e.g., Chinese input → Chinese story).
    """


    storyboard = generate_scientific_storyboard(user_input)

    prompt = f"""
You are writing an educational comic-style story (like an episode of "Cells at Work!")
that shows what happens inside the human body during the user's activities.

You are given a 4-scene scientific storyboard. Turn it into a vivid narrative.

Perspective:
- You may use third-person narration, or choose a main character such as:
  - a specific cell type (e.g., neutrophil, hepatocyte, neuron),
  - a molecule (e.g., caffeine, glucose),
  - or a small team (e.g., "Energy Squad").
- The perspective does NOT have to be first-person "I".
- Choose the perspective that best fits the main mechanisms and keep it consistent.

Language:
- Use the SAME primary language as the user's input.
- If the user's input is mainly Chinese, write the story in Chinese.
- If it is mainly English, write the story in English.
- It is okay to keep technical terms (like enzyme names) in English if needed.

STRUCTURE (MANDATORY):
Keep EXACTLY these four headings, in this exact form:

**SCENE 1: ENTRY**
**SCENE 2: TRANSPORT**
**SCENE 3: THE MECHANISM (CLIMAX)**
**SCENE 4: RESOLUTION**

Under each heading, write 2–5 paragraphs of flowing narrative.

STYLE RULES:
- Rich, concrete biological details (organs, cells, receptors, enzymes, pathways).
- Anime/comic style: dramatic, playful, but scientifically plausible.
- Use dialogues between characters (cells, molecules, organs, pathogens, etc.)
  when it helps make mechanisms vivid.

User input (for language and vibe):
\"\"\"{user_input}\"\"\"

Here is the storyboard you must follow (this is a plan, not to be copied verbatim):

{storyboard}

Now write the full story:
"""

    model = genai.GenerativeModel(TEXT_MODEL)
    response = model.generate_content(prompt)
    story = response.text.strip()
    return story


# ========== 4. Comic Panels: Split Full Story into Scene-Level Panels ==========


def build_comic_scenes_from_story(story: str, n_panels: int):
    """
    Input:
        - story: the full generated narrative
        - n_panels: desired number of comic panels

    Output:
        A list of length n_panels, where each element is a dict:
            {
                "scene": "1–3 sentences describing what happens in this panel",
                "narration": "optional narration text"
            }
    """

    model = genai.GenerativeModel(TEXT_MODEL)

    prompt = f"""
You are turning a science story into a {n_panels}-panel comic.

For each panel, write:
- "scene": 1–3 sentences describing what is happening in this panel (visually).
- "narration": 1 short narration sentence that could appear in a narration box.

Rules:
- EXACTLY {n_panels} panels.
- Panels must follow the original story order.
- Use the SAME language as the STORY (Chinese if Chinese, otherwise English).
- Each "scene" should be short and focused on one key moment.

Return ONLY valid JSON in the following format:
[
  {{
    "scene": "...",
    "narration": "..."
  }},
  ...
]

STORY:
{story}
"""

    response = model.generate_content(
        prompt,
        generation_config=genai_types.GenerationConfig(
            temperature=0.4,
            response_mime_type="application/json"
        )
    )

    try:
        panels = json.loads(response.text)
    except Exception:
        # If JSON parsing fails, fallback: split the full story evenly into n_panels segments.
        chunks = re.split(r'\n\s*\n+', story.strip())
        if len(chunks) < n_panels:
            chunks = chunks + [""] * (n_panels - len(chunks))
        size = max(1, len(chunks) // n_panels)
        panels = []
        for i in range(n_panels):
            seg = "\n\n".join(chunks[i*size:(i+1)*size]).strip()
            panels.append({"scene": seg, "narration": ""})

    if len(panels) > n_panels:
        panels = panels[:n_panels]
    elif len(panels) < n_panels:
        while len(panels) < n_panels:
            panels.append({"scene": "", "narration": ""})

    return panels


# ========== 5. OCR & scientific-term correction helper functions (reuse your original logic) ==========


def extract_text_from_image(img):
    """Extract text from an image using OCR."""
    resp = image_client.models.generate_content(
        model=TEXT_MODEL,
        contents=[
            "Extract ALL readable text in this image. Do NOT correct the text. Keep exact spelling.",
            img
        ]
    )
    return resp.text


def extract_words(text):
    """Extract a list of words from the text."""
    words = re.findall(r'\b[a-zA-Z]+\b', text.lower())
    return words


def map_corrections(ocr_words, corrected_words):
    """Map OCR-misrecognized words to their corrected forms."""
    correction_map = {}
    for i, ocr_word in enumerate(ocr_words):
        if i < len(corrected_words):
            if ocr_word != corrected_words[i]:
                correction_map[ocr_word] = corrected_words[i]
    return correction_map


def load_science_terms():
    """Load the scientific terminology dictionary (extendable)."""
    basic_terms = [
        "ATP", "GLUT4", "insulin", "glucose", "mitochondria", "mitochondrion",
        "enzyme", "protein", "carbohydrate", "lipid", "amino acid",
        "stomach", "intestine", "bloodstream", "liver", "kidney",
        "muscle", "cell", "tissue", "organ", "amylase", "digestion"
    ]
    return [t.lower() for t in basic_terms]


def correct_text_with_science(ocr_text, science_terms):
    """Correct spelling errors in OCR text, including scientific terms."""
    model = genai.GenerativeModel(TEXT_MODEL)

    prompt = f"""
You are a professional comic text proofreader.
Your tasks:

1. Fix ALL spelling errors in the text (general English vocabulary included).
2. If a word seems scientific, correct it using ONLY the following dictionary:
{', '.join(science_terms)}
3. Produce clean, natural comic-style dialogue.
4. Keep meaning consistent. Do NOT invent scientific terms.

OCR text:
{ocr_text}

Return ONLY the corrected text.
"""
    resp = model.generate_content(prompt)
    return resp.text


def generate_corrected_image(story, corrected_text, n_images=1):
    """Regenerate the image using the corrected text."""
    refine_prompt = f"""
Create a colorful, cartoon-style biology comic panel.
Scene:
{story}

Use EXACTLY the following corrected text in speech bubbles:
\"\"\"
{corrected_text}
\"\"\"

Rules:
- NO additional text besides what is given.
- Use clean, printed sans-serif text (Arial or similar).
- Ensure ALL spellings match exactly.
- Keep characters expressive and cartoon-like.
- Make text clearly readable with high contrast.
"""

    resp = image_client.models.generate_content(
        model=IMAGE_MODEL,
        contents=[refine_prompt],
        config={"candidate_count": n_images}
    )

    images = []
    for cand in resp.candidates:
        for part in cand.content.parts:
            if getattr(part, "inline_data", None):
                images.append(PilImage.open(BytesIO(part.inline_data.data)))

    return images


# ========== 6. Generate illustrations from the story (following your original logic) ==========

def generate_images_from_story(story: str, n_images: int = 1, use_ocr_correction: bool = True):
    """
    Generate `n_images` comic illustrations from the story, with optional OCR correction.
    """
    panels = build_comic_scenes_from_story(story, n_images)

    images = []
    panel_summaries = []

    if use_ocr_correction:
        science_terms = load_science_terms()

    max_retries = 3

    for panel_idx, panel in enumerate(panels):
        panel_scene = panel.get("scene", "").strip()
        if not panel_scene:
            panel_scene = story

        image_generated = False
        retry_count = 0

        while not image_generated and retry_count < max_retries:
            retry_count += 1

            prompt = f"""
You are a Japanese anime illustrator. Create a single-page comic-style panel based on the story below.

Style:
- Cute 2D anime style (NOT 3D)
- Soft pastel colors, clean line art, warm and friendly mood
- Anthropomorphic characters (cells, organs, molecules as little anime characters)
- Clear main character and simple composition

Comic layout:
- Draw several main characters clearly.
- Next to or above each main character, add a tiny label with their name or role
  (for example: "Muscle cell", "Ibuprofen", "Inflammation", "White blood cell", etc.).
- Add one small narration box in a corner that summarizes the scene in ONE short sentence.
- If there is dialogue in the story, add 1–3 simple speech bubbles with SHORT phrases
  (like "We must repair the muscle!" / "I will stop the inflammation!"), not long paragraphs.

Text:
- Use the same language as the STORY (Chinese if the story is in Chinese, otherwise English).
- Keep all text very short and clear, like in a children's science comic.

STORY:
{panel_scene}
"""

            try:
                resp = image_client.models.generate_content(
                    model=IMAGE_MODEL,
                    contents=[prompt],
                )

                raw_img = None
                if resp and resp.candidates:
                    for c in resp.candidates:
                        if c.content and c.content.parts:
                            for p in c.content.parts:
                                if getattr(p, "inline_data", None) is not None:
                                    raw_img = PilImage.open(BytesIO(p.inline_data.data))
                                    break
                        if raw_img:
                            break

                if not raw_img:
                    if retry_count < max_retries:
                        print(f"[Warning] Image {panel_idx+1} failed to generate, retrying {retry_count}/{max_retries}...")
                        continue
                    else:
                        print(f"[Error] Image {panel_idx+1} failed to generate — maximum retry attempts reached.")
                        placeholder = PilImage.new('RGB', (800, 600), color='lightgray')
                        draw = ImageDraw.Draw(placeholder)
                        try:
                            font = ImageFont.truetype("/usr/share/fonts/truetype/liberation/LiberationSans-Regular.ttf", 40)
                        except:
                            font = ImageFont.load_default()
                        text = f"Image {panel_idx+1} Failed to Generate"
                        draw.text((200, 250), text, fill='black', font=font)
                        raw_img = placeholder

                final_img = raw_img
                if use_ocr_correction and raw_img:
                    try:
                        ocr_text = extract_text_from_image(raw_img)
                        corrected_text = correct_text_with_science(ocr_text, science_terms)

                        refine_prompt = f"""
{prompt}

IMPORTANT: Use EXACTLY these corrected text labels and dialogue in your image:
\"\"\"
{corrected_text}
\"\"\"
Ensure all text spellings match exactly as provided above.
"""

                        resp2 = image_client.models.generate_content(
                            model=IMAGE_MODEL,
                            contents=[refine_prompt],
                        )

                        if resp2 and resp2.candidates:
                            for c in resp2.candidates:
                                if c.content and c.content.parts:
                                    for p in c.content.parts:
                                        if getattr(p, "inline_data", None) is not None:
                                            final_img = PilImage.open(BytesIO(p.inline_data.data))
                                            break
                                if final_img != raw_img:
                                    break

                    except Exception as e:
                        print(f"OCR correction failed: {e}")
                        final_img = raw_img

                images.append(final_img)
                panel_summaries.append(panel_scene)
                image_generated = True

            except Exception as e:
                print(f"[Error] Exception occurred while generating image {panel_idx+1}: {e}")
                if retry_count >= max_retries:
                    placeholder = PilImage.new('RGB', (800, 600), color='lightgray')
                    draw = ImageDraw.Draw(placeholder)
                    try:
                        font = ImageFont.truetype("/usr/share/fonts/truetype/liberation/LiberationSans-Regular.ttf", 40)
                    except:
                        font = ImageFont.load_default()
                    text = f"Error: {str(e)[:50]}"
                    draw.text((50, 250), text, fill='red', font=font)
                    images.append(placeholder)
                    panel_summaries.append(panel_scene)
                    image_generated = True

    if len(images) != n_images:
        print(f"[Warning] The number of generated images ({len(images)}) does not match the requested amount ({n_images}).")

    return images, panel_summaries


# ========== 7. Story summary for each panel ==========

def shorten_summary(text: str, max_sentences: int = 3) -> str:
    """
    Compress a scene description into at most `max_sentences` sentences.
    Also remove any leading title lines such as 'SCENE 1: ENTRY ...'.
    """

    if not text:
        return ""

    text = text.strip()
    text = re.sub(r'^SCENE\s*\d+\s*:[^\n]*\n', '', text, flags=re.IGNORECASE)

    parts = re.split(r'(?<=[。！？.!?])\s+', text)
    parts = [p.strip() for p in parts if p.strip()]

    if not parts:
        return ""

    short = " ".join(parts[:max_sentences])

    if not short:
        short = text[:120] + ("..." if len(text) > 120 else "")

    return short


# ========== 8. Streamlit UI ==========

st.title("🧬 BodyBuddies Adventures 🧬")

st.markdown("""
Turn your everyday activities into fun scientific stories!
Just tell us what you did today — the AI will use RAG + multi-organ physiology
to build a 4-scene internal adventure, then generate matching illustrations.
""")

user_activity = st.text_input(
    "Describe your daily activity:",
    value="This morning I drank coffee on an empty stomach, had fried chicken for lunch, and went swimming in the afternoon.",
    help="Tell us anything you did today — the AI will turn it into a scientific adventure happening inside your body!"
)

col1, col2, col3 = st.columns(3)
with col1:
    n_images = st.slider("How many illustrations to generate?", 1, 6, 1)
with col2:
    use_ocr = st.checkbox(
        "Enable OCR spell-correction",
        value=True,
        help="Uses OCR to fix text inside the images for better accuracy"
    )
with col3:
    st.info("💡 Tip: OCR makes text more accurate but may take a bit longer.")

if st.button("✨ Generate Story & Illustrations", type="primary"):
    if not user_activity.strip():
        st.warning("Please enter a description first!")
    else:
        # Generate RAG-based physiology story
        with st.spinner("Crafting your 4-scene internal science adventure..."):
            story_result = generate_physiology_story(user_activity)

        st.subheader("📖 Final Story")
        st.markdown(story_result)

        # Generate illustrations
        with st.spinner("Creating illustrations..."):
            imgs, panel_summaries = generate_images_from_story(
                story_result,
                n_images=n_images,
                use_ocr_correction=use_ocr
            )

        if imgs:
            st.subheader("🖼️ Generated Illustrations")
            for i, (img, summary) in enumerate(zip(imgs, panel_summaries), 1):
                st.image(img, caption=f"Illustration {i}", width="content")
                short = shorten_summary(summary, max_sentences=5)
                st.markdown(f"**What this panel shows:** {short}")
        else:
            st.info("📘 The story was generated successfully! (Illustration generation may be temporarily unavailable.)")

st.markdown("---")
st.markdown("""
<div style='text-align: center; color: gray; font-size: 0.8em;'>
    Powered by Google Gemini API | BodyBuddies Adventures vRAG
</div>
""", unsafe_allow_html=True)



# New Section

In [ ]:
import os

NGROK_TOKEN = os.getenv("NGROK_AUTHTOKEN")

if NGROK_TOKEN is None:
    raise ValueError("❌ NGROK_AUTHTOKEN not found. Please set it in Colab Secrets!")

os.environ["NGROK_AUTHTOKEN"] = NGROK_TOKEN


In [ ]:
!pip install pyngrok

In [ ]:
!streamlit run app.py --server.port 8501 &>/content/logs.txt &

In [ ]:
from pyngrok import ngrok
ngrok.kill()

In [ ]:

from pyngrok import ngrok

public_url = ngrok.connect(8501)
public_url
